## **IRF5 Region Association Analysis with SLE in UK Biobank WGS**
This notebook extracts variants in the IRF5 gene region (±1Mb) from UK Biobank WGS data and tests for association with SLE using logistic regression.

### **Step 1: Install PLINK2**

In [ ]:
# Install PLINK2
wget https://s3.amazonaws.com/plink2-assets/alpha6/plink2_linux_avx2_20251019.zip
unzip -o plink2_linux_avx2_20251019.zip
chmod a+x plink2 # Make PLINK2 executable
./plink2 --version

### **Step 2: Set Variables**
- IRF5 genomic location (GRCh38.p14): chr7:128,937,032-128,950,038

In [ ]:
# List project folders to verify file paths
ls /mnt/project

In [ ]:
## Set IRF5 gene region variables
CHR=7
GENE_START=128937032
GENE_END=128950038
FLANK=1000000

# Calculate region with flanking sequences
REGION_START=$((GENE_START - FLANK))
REGION_END=$((GENE_END + FLANK))

## Set working directory & file path variables for input data
BGEN_DIR="/mnt/project/Bulk/DRAGEN WGS/DRAGEN population level WGS variants, BGEN format [500k release]"
PHENO_FILE="/mnt/project/02.Phenotype_SampleQC/sle_gwas.txt"
KEEP_FILE="/mnt/project/02.Phenotype_SampleQC/sample_ids_qc_pass.txt"
EXTRACT_FILE="/mnt/project/03.Variant_QC/WGS_QC/ukb24309_c7_b0_v1_qc_pass.snplist"

In [ ]:
# Inspect sample ID format in the BGEN sample metadata file
head -n 10 "${BGEN_DIR}/ukb24309_c${CHR}_b0_v1.sample"

In [ ]:
# Inspect sample ID format in WGS QC file - should match BGEN
head -n 10 ${KEEP_FILE}

In [ ]:
# Inspect QC Pass SNPs in WGS QC file
head -n 10 ${EXTRACT_FILE}

In [ ]:
# Inspect phenotypes files
head -n 10 ${PHENO_FILE}

### **Step 3: Extract IRF5 Region (Only QC-Passed Variants & Samples)**

In [ ]:
./plink2 \
  --bgen "${BGEN_DIR}/ukb24309_c${CHR}_b0_v1.bgen" ref-last \  # WGS data has last allele as ref
  --sample "${BGEN_DIR}/ukb24309_c${CHR}_b0_v1.sample" \ 
  --chr ${CHR} \
  --from-bp ${REGION_START} \
  --to-bp ${REGION_END} \
  --extract ${EXTRACT_FILE} \  # Include only QC-passed variants
  --keep ${KEEP_FILE} \  # Include only QC-passed samples
  --force-intersect \
  --make-pgen \  # Create PLINK2 binary format (.pgen/.pvar/.psam)
  --out irf5_region_qc # Output file prefix


echo "Extracted region: chr${CHR}:${REGION_START}-${REGION_END}"

### **Step 4: Test Extracted Variants For Association With SLE Using Logistic Regression Model**

In [ ]:
./plink2 \
  --pfile irf5_region_qc \  # Input PLINK2 binary files from Step 3
  --glm firth firth-residualize hide-covar \  # Logistic regression: use Firth correction, residualize, hide covariate stats
  --pheno ${PHENO_FILE} \
  --pheno-name has_sle_icd10 \
  --1 \  # Recode case status: 1=case (PLINK default is 2=case)
  --covar ${PHENO_FILE} \
  --covar-name sex age ethnic_group pc1 pc2 pc3 pc4 pc5 pc6 pc7 pc8 pc9 pc10 \
  --covar-variance-standardize \  # Standardise covariates to mean=0, variance=1
  --mac 20 \  # Minimum minor allele count threshold
  --threads 16 \
  --out irf5_sle  # Output file prefix

In [ ]:
# View first 20 results
head -n 21 irf5_sle.has_sle_icd10.glm.firth

# Expected columns: CHROM POS ID REF ALT A1 OMITTED A1_FREQ TEST OBS_CT OR LOG(OR)_SE Z_STAT P ERRCODE

In [ ]:
# Count variants with P < 5.7e-6 (Bonferroni-corrected significance threshold)
awk 'NR>1 && $15 < 5.7e-6' irf5_sle.has_sle_icd10.glm.firth | wc -l # NR>1: skip header line; $15: P-value column 

In [ ]:
# Save all results sorted by p-value as CSV
(head -n 1 irf5_sle.has_sle_icd10.glm.firth | tr '\t' ',' && \  # Extract header, convert tabs to commas
 tail -n +2 irf5_sle.has_sle_icd10.glm.firth | \  # Get all data lines (skip header)
 sort -g -k15,15 | \  # Sort by column 15 (P-value) in general numeric order
 tr '\t' ',') > irf5_sle_results_sorted.csv  # Convert tabs to commas, save to CSV

### **Step 5: Annotate Top 100 Variants With rsID And Nearest Gene**

In [ ]:
# Create Python script to annotate top 100 variants with rsIDs
cat > get_top100_rsids.py << 'EOF'
import pandas as pd
import requests # To make internet requests
import time # To add delays

# Read only top 100 variants (already sorted by p-value)
df = pd.read_csv('irf5_sle_results_sorted.csv', nrows=100)

print(f"Processing top 100 variants by p-value...")

# Define function to make API request to get rsID based on variant chr & pos
def get_rsid(chrom, pos):
    url = f"https://rest.ensembl.org/overlap/region/human/{chrom}:{pos}-{pos}?feature=variation"
    try:
        response = requests.get(url, headers={"Content-Type": "application/json"}, timeout=10) # Want JSON format, give up if >10s
        if response.ok:
            data = response.json()
            if data and len(data) > 0: # If got results and there's at least 1 variant
                rsid = data[0].get('id', 'NA') # Get 'id' field of first result, if not 'NA'
                return rsid
        return 'NA'
    except:
        return 'NA'

rsids = [] # Empty list to store results
for idx, row in df.iterrows(): #idx (starts from 0) is the row number, row is the data within
    chrom = row['#CHROM']
    pos = row['POS']
    rsid = get_rsid(chrom, pos)
    rsids.append(rsid)
    
    if (idx + 1) % 10 == 0: 
        print(f"Processed {idx+1}/100 variants...")
    
    time.sleep(0.2) # Only 5 req/s to avoid overwhelming Ensembl API

df['rsID'] = rsids # Create new column for rsIDs

# Reorder columns
cols = df.columns.tolist() # Get list of all column names
id_idx = cols.index('ID')  # Find position of 'ID' column
cols.insert(id_idx + 1, cols.pop(cols.index('rsID'))) # Move rsID to be right after ID
df = df[cols] # Apply new column order

# Save
df.to_csv('irf5_top100_with_rsids.csv', index=False)

# Summary
total_rsids = sum(1 for r in rsids if r != 'NA' and r.startswith('rs'))
print(f"\n=== Completed ===")
print(f"Found {total_rsids} rsIDs out of 100 variants")
print("\nTop 20 variants:")
print(df[['#CHROM', 'POS', 'ID', 'rsID', 'OR', 'P']].head(20))
EOF

# Run script
python3 get_top100_rsids.py

In [ ]:
# Create Python script to annotate variants with genes
cat > annotate_genes.py << 'EOF'
import pandas as pd
import requests
import time

# Read rsID-annotated top 100 results
df = pd.read_csv('irf5_top100_with_rsids.csv')

print(f"Annotating {len(df)} variants with gene names...")

def get_gene_annotation(chrom, pos, ref, alt):
    """
    Query Ensembl VEP (Variant Effect Predictor) to determine if variant is inside a gene.
    Returns gene name if variant is in a gene, 'regulatory_region' if in regulatory area,
    or 'intergenic' if between genes.
    """
    url = "https://rest.ensembl.org/vep/human/region"
    
    # Format variant location as: chr:start-end:strand/ref/alt
    # Example: "7:129013434-129013434:1/G/A"
    region = f"{chrom}:{pos}-{pos}:1/{ref}/{alt}"
    
    params = {
        "variant_class": "1"  # Request variant classification information
    }
    
    try:
        # Query VEP API for variant annotation
        response = requests.get(
            f"{url}/{region}",
            headers={"Content-Type": "application/json"},
            params=params,
            timeout=10
        )
        
        if response.ok:
            data = response.json()  # Parse JSON response
            if data and len(data) > 0:
                # Extract gene symbols from transcript consequences
                genes = set()  # Use set to avoid duplicates
                if 'transcript_consequences' in data[0]:
                    # Loop through all affected transcripts
                    for consequence in data[0]['transcript_consequences']:
                        if 'gene_symbol' in consequence:
                            genes.add(consequence['gene_symbol'])  # Collect unique gene names
                
                if genes:
                    # If variant affects multiple genes, join with commas
                    return ','.join(sorted(genes))
                
                # Check for intergenic consequences (variant between genes)
                if 'intergenic_consequences' in data[0]:
                    for consequence in data[0]['intergenic_consequences']:
                        if 'consequence_terms' in consequence:
                            if consequence.get('consequence_terms') == ['intergenic_variant']:
                                # Placeholder for future nearest gene extraction from VEP
                                pass
                
                # Check if variant is in a regulatory region (enhancer, promoter, etc.)
                if 'regulatory_feature_consequences' in data[0]:
                    return 'regulatory_region'
                    
        return 'intergenic'  # Not in any gene or regulatory region
    except Exception as e:
        return 'error'  # Return error if API call fails

def get_nearest_gene(chrom, pos):
    """
    For intergenic variants, find the nearest protein-coding gene within ±500kb.
    Returns formatted string with gene name, distance, and direction.
    """
    # Search 500kb flanking variant
    window = 500000
    start = pos - window
    end = pos + window
    
    # Query Ensembl for all genes in the window
    url = f"https://rest.ensembl.org/overlap/region/human/{chrom}:{start}-{end}"
    params = {
        "feature": "gene",  # Only return genes
        "biotype": "protein_coding"  # Only protein-coding genes
    }
    
    try:
        response = requests.get(
            url,
            headers={"Content-Type": "application/json"},
            params=params,
            timeout=10
        )
        
        if response.ok:
            genes = response.json()  # List of all genes in the window
            if genes:
                # Find gene with minimum distance to variant
                min_distance = float('inf')  # Start with infinity
                nearest = None
                nearest_direction = None
                
                for gene in genes:
                    gene_start = gene.get('start', 0)
                    gene_end = gene.get('end', 0)
                    gene_name = gene.get('external_name', gene.get('id', 'unknown'))
                    
                    # Calculate distance and direction from variant to gene
                    if pos < gene_start:
                        # Variant is upstream of gene
                        distance = gene_start - pos
                        direction = "upstream"
                    elif pos > gene_end:
                        # Variant is downstream of gene
                        distance = pos - gene_end
                        direction = "downstream"
                    else:
                        # Variant is inside the gene (but shouldn't happen for intergenic variants)
                        distance = 0
                        direction = "in"
                    
                    # Track the closest gene
                    if distance < min_distance:
                        min_distance = distance
                        nearest = gene_name
                        nearest_direction = direction
                
                # Format output with distance and direction
                if nearest and min_distance > 0:
                    # Convert base pairs to kilobases (2 dp)
                    return f"{nearest} ({min_distance/1000:.2f}kb {nearest_direction})"
                elif nearest:
                    # If distance is 0 (variant inside gene), just return gene name
                    return nearest
        
        return 'intergenic'  # No genes found in window
    except Exception as e:
        return 'intergenic'  # Return intergenic if API call fails

# Annotate all variants with gene information
genes = []
for idx, row in df.iterrows():
    chrom = row['#CHROM']  # Chromosome number
    pos = row['POS']  # Position on chromosome
    ref = row['REF']  # Reference allele
    alt = row['ALT']  # Alternate allele
    
    # Step 1: Check if variant is IN a gene using VEP
    gene = get_gene_annotation(chrom, pos, ref, alt)
    
    # Step 2: If variant is intergenic, find nearest gene
    if gene == 'intergenic':
        gene = get_nearest_gene(chrom, pos)
        time.sleep(0.3)  # Additional delay for second API call (600ms total per intergenic variant)
    
    genes.append(gene)
    
    # Progress update every 10 variants
    if (idx + 1) % 10 == 0:
        print(f"Processed {idx+1}/{len(df)} variants... (last gene: {gene})")
    
    time.sleep(0.3)  # 300ms delay between requests to avoid overwhelming API

# Add Gene column to dataframe
df['Gene'] = genes

# Reorder columns to place Gene right after rsID for better readability
cols = df.columns.tolist()  # Get list of all column names
rsid_idx = cols.index('rsID')  # Find position of rsID column
cols.insert(rsid_idx + 1, cols.pop(cols.index('Gene')))  # Move Gene to be after rsID
df = df[cols]  # Apply new column order to df

# Save annotated results
df.to_csv('irf5_top100_annotated.csv', index=False)

# Print summary statistics
print(f"\n=== Completed ===")
print(f"\nSaved to: irf5_top100_annotated.csv")
print("\nTop 20 variants with genes:")
print(df[['POS', 'rsID', 'Gene', 'OR', 'P']].head(20))  # Display key columns for top 20 variants
EOF

# Execute the Python script
python3 annotate_genes.py